In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays 
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
# include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
Params = get_default_params_PLD()
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [ ]:
S_train_list = Params["S_train_all"] # training data size
is_ridge = Params["is_ridge"] # whether to use ridge regression
S_train_list = [100] 

2

In [4]:
Random.seed!(seed)
is_Wang_Qi_Shen = true;
is_same_util_para = true;
if is_Wang_Qi_Shen
    project_dir = "Model_Mis_Wang_Qi_Shen_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    project_dir = "Model_Mis_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")

data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Codes/Data/Product_Line_Design/Model_Mis_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/


In [5]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [6]:
function output_results(S_train,lambda,data_dir,instances,fig_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),obj True:",round.(mean(obj_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj ETO:",round.(mean(obj_ETO),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),profit True:",round.(mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))
    # fig_name = string(data_dir, "RPLD_vs_ETOPLD_S=$(S_train)_lambda=$lambda.pdf")
    # include_std = false
    # line_plot_RPLD_vs_ETOPLD(profit_ETO,profit_RO,gamma_list,include_std,fig_name,fig_display)
    return profit_True, profit_ETO, profit_RO
end

output_results (generic function with 1 method)

In [7]:
function plot_box(S_train,lambda,data_dir,chosen_indices,is_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),obj True:",round.(mean(obj_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj ETO:",round.(mean(obj_ETO),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),profit True:",round.(mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))

    Profit_ETO_All_Ins = profit_ETO./mean(profit_ETO)
    Profit_RO_All_Ins = Dict();
    for g_index in 1:length(gamma_list)
        gamma=gamma_list[g_index]
        Profit_RO_All_Ins["gamma=$(gamma)"] = profit_RO[:,g_index]./mean(profit_ETO)
    end
    
    gamma_chosen = gamma_list[chosen_indices]
    data = [Profit_ETO_All_Ins, [Profit_RO_All_Ins["gamma=$(gamma)"] for gamma in gamma_chosen]...]
    labels = ["ETO"; ["RO($gamma)" for gamma in gamma_chosen]]
    fig_name = string(data_dir, "Boxplot_RPLD_vs_ETOPLD_S=$(S_train)_lambda=$(lambda)_Normalize.pdf")
    boxplot_RPLD_vs_ETOPLD(data,labels,fig_name,is_display)

end

plot_box (generic function with 1 method)

In [ ]:
function Generate_Wang_Qi_Max_True_Data_Mis(N_x, N_u, n_sample, N_Max,theta_true,u_lb,u_ub,Mis_coef)
    # 初始化存储
    X = Vector{Matrix{Float64}}(undef, n_sample); # n 个 m×d 矩阵
    Z = Matrix{Float64}(undef, n_sample, N_u);      # n×p 矩阵
    Y = Vector{Int}(undef, n_sample);             # n 维向量

    for i in 1:n_sample
        z_i = rand(Uniform(u_lb, u_ub), N_u)
        Z[i, :] = z_i

        X_i = zeros(N_Max, N_x)
        for j in 1:N_Max
            # X_i[j, :] = rand(Uniform(0.0, 1.0), d)
            X_i[j, :] = rand(0.0:1.0, N_x)
        end
        X[i] = X_i
        # --- 计算选择概率 ---
        V0 = 1.0
        utilities = zeros(N_Max)
        for j in 1:N_Max
            x_ij = X_i[j, :] # 第 j 个产品的设计
            # 计算效用 Uz(x_ij) = α₀* + <α*, x_ij> + <β*, z_i> + x_ij^T * A* * z_i
            utility = theta_true.alpha0 +
                        dot(theta_true.alpha, x_ij) +
                        dot(theta_true.beta, z_i) +
                        dot(x_ij, theta_true.A * z_i) # x_ij^T * A* * z_i
            utilities[j] = utility
        end

        # 计算分子 exp(Uz(x_ij))
        exp_utilities = exp.(utilities).^(Mis_coef)
        # 计算分母 (V0 + sum(exp(Uz(x_il))))
        denominator = V0 + sum(exp_utilities)

        # 计算选择每个产品 j 的概率
        prob_choose_product = exp_utilities ./ denominator
        # 计算选择默认选项 (索引 0) 的概率
        prob_choose_default = V0 / denominator

        # 构建完整的概率向量 [P(选择默认), P(选择产品1), ..., P(选择产品m_actual)]
        choice_probs = vcat(prob_choose_default, prob_choose_product)
        # 选择结果: 0 表示默认选项, 1 表示第一个产品, ..., m_actual 表示第 m_actual 个产品
        y_i = sample(0:N_Max, Weights(choice_probs))
        Y[i] = y_i
    end
    return X,Y,Z
end

Generate_Wang_Qi_Max_True_Data_Mis (generic function with 1 method)

In [9]:
function Generate_Data_this_Same_Para_Mis(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed,Mis_coef)
    theta_true = theta_true_Fixed
    r_params = r_params_Fixed
    u_lb = -0.1
    u_ub = 0.1
    X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data_Mis(N_x, N_u, S_train, N_Max,theta_true,u_lb,u_ub,Mis_coef)
    X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data_Mis(N_x, N_u, S_test, N_Max,theta_true,u_lb,u_ub,Mis_coef);
    Input_Data = Dict()
    Input_Data["theta_true"] = theta_true
    Input_Data["r_params"] = r_params
    Input_Data["X_train"] = X_train
    Input_Data["Y_train"] = Y_train
    Input_Data["Z_train"] = Z_train
    Input_Data["X_test"] = X_test
    Input_Data["Y_test"] = Y_test
    Input_Data["Z_test"] = Z_test

    asorrtment_train = Array{Vector{Int64}}(undef,S_train)
    for s in 1:S_train
        asorrtment_train[s] = collect(1:N_Max)
    end
    Input_Data["asorrtment_train"] = asorrtment_train
    return Input_Data
end

Generate_Data_this_Same_Para_Mis (generic function with 1 method)

In [10]:
function calculate_profit_Mis(params, r0, r, X_val, z_input,Mis_Coef)
    alp0 = params.alpha0
    alp = params.alpha
    beta = params.beta
    A = params.A

    V0 = 1.0
    utilities = zeros(N)
    for j in 1:N
        x_ij = X_val[j, :] # 第 j 个产品的设计
        # 计算效用 Uz(x_ij) = α₀* + <α*, x_ij> + <β*, z_i> + x_ij^T * A* * z_i
        utility = alp0 + dot(alp, x_ij) + dot(beta, z_input) + dot(x_ij, A * z_input) # x_ij^T * A* * z_i
        utilities[j] = utility^(Mis_Coef)
    end

    exp_utilities = exp.(utilities)
    # 计算分母 (V0 + sum(exp(Uz(x_il))))
    denominator = V0 + sum(exp_utilities)

    # 计算选择每个产品 j 的概率
    prob_choose_product = exp_utilities ./ denominator
    # 计算选择默认选项 (索引 0) 的概率
    prob_choose_default = V0 / denominator

    profits = r0 .+ X_val * r 
    total_profit = profits' * prob_choose_product
    return total_profit
end

calculate_profit_Mis (generic function with 1 method)

In [11]:
function solve_ETO_This_Mis(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,Mis_Coef)
    r0 = r_params.r0; r = r_params.r;

    RST_Dict = Dict()
    obj_list = zeros(S_test); x_list = zeros(S_test,N,N_x); time_list = zeros(S_test); profit_list = zeros(S_test); status_list = Vector{String}(undef, S_test)
    status_all = "OPTIMAL"
    for i in 1:S_test
        z_input = Z_test[i,:];
        nu0, nu = compute_w(theta_Input,z_input)
        obj_list[i], x_list[i,:,:], time_list[i],status_list[i] = ETO_PLD(N,N_x,nu0, nu, r0, r,c_l,d_r,rev_gap,num_c,Time_Limit);
        if status_list[i] != "OPTIMAL"
            status_all = status_list[i]
            println("Warning: The optimization for test instance $i did not reach optimality. Status: ", status_list[i])
            break
        end
        profit_list[i] = calculate_profit_Mis(theta_true, r0, r, x_list[i,:,:], z_input,Mis_Coef)                       
    end
    RST_Dict["obj"] = obj_list
    RST_Dict["X"] = x_list
    RST_Dict["time"] = time_list
    RST_Dict["profit"] = profit_list
    RST_Dict["status"] = status_list
    return RST_Dict,status_all
end

solve_ETO_This_Mis (generic function with 1 method)

In [12]:
function solve_RO_this_Mis(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub,Mis_Coef)
    r0 = r_params.r0; r = r_params.r;
    RST_Dict = Dict();
    
    obj_list = zeros(S_test); x_list = zeros(S_test,N,N_x); time_list = zeros(S_test); profit_list = zeros(S_test); status_list = Vector{String}(undef, S_test)
    status_all = "OPTIMAL"
    for i in 1:S_test
        z_input = Z_test[i,:];                
        nu0, nu = compute_w(theta_Input,z_input)
        obj_list[i], x_list[i,:,:], time_list[i],status_list[i] = RO_PLD(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit)
        if status_list[i] != "OPTIMAL"
            status_all = status_list[i]
            println("Warning: The RO optimization for test instance $i did not reach optimality. Status: ", status_list[i])
            break
        end
        profit_list[i] = calculate_profit_Mis(theta_true, r0, r, x_list[i,:,:], z_input,Mis_Coef)
    end
    RST_Dict["obj"] = obj_list
    RST_Dict["X"] = x_list
    RST_Dict["time"] = time_list
    RST_Dict["profit"] = profit_list
    RST_Dict["status"] = status_list
    return RST_Dict,status_all
end

solve_RO_this_Mis (generic function with 1 method)

In [13]:
function main_process(S_train_list,lambda,instances,Mis_coef)
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    config = Dict()
    config["theta_true_Fixed"] = theta_true_Fixed
    config["r_params_Fixed"] = r_params_Fixed
    save(string(data_dir, "config_Mis_coef=$(Mis_coef)_lambda=$(lambda).jld2"), config);

    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data = Dict()
        RST_True_All = Dict()
        RST_ETO_All = Dict()
        RST_RO_All = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Generate_Data_this_Same_Para_Mis(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed,Mis_coef)
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            nu0_true,nu_true = compute_w(theta_true,Z_test[1,:])  
            nu_all_true = [nu0_true;nu_true]
            # ******** Estimation *************
            theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Asorrtment_train,is_ridge, lambda)
            nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  
            nu_all_hat = [nu0_hat;nu_hat]


            # --- 过滤异常情况 ---
            if any(isnan, nu_all_hat)
                println("Estimate contains NaN values.")
                continue
            end
            # if norm(vec(nu_all_true .- nu_all_hat),2) >= norm_bounds
            #     println("Estimate is too far from true parameters.")
            #     continue
            # end
            Input_Data_this["theta_hat"] = theta_hat
            Input_Data_this["nu_true"] = nu_all_true
            Input_Data_this["nu_hat"] = nu_all_hat
            Input_Data["ins=$(ins)"] = Input_Data_this

            # ******** True Model *************
            theta_Input = theta_true
            RST_True,status_True = solve_ETO_This_Mis(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,Mis_coef)
            # println("Oracle: status = ",status_True,",obj=",RST_True["obj"][1])
            if status_True != "OPTIMAL"
                println("Warning: The true model did not reach optimality")
                continue
            end
            RST_True_All["ins=$(ins)"] = RST_True

            # ******** ETO Model *************
            RST_ETO,status_ETO = solve_ETO_This_Mis(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,Mis_coef)
            # println("ETO: status = ",status_ETO,",obj=",RST_ETO["obj"][1])
            if status_ETO != "OPTIMAL"
                println("Warning: The ETO model did not reach optimality")
                continue
            end
            RST_ETO_All["ins=$(ins)"] = RST_ETO
            
            # ******** RO Model *************
            RST_RO_this = Dict()
            gamma = gamma_list[1]
            RST_RO,status_RO = solve_RO_this_Mis(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub,Mis_coef)
            # println("gamma = $gamma, RO: status = ",status_RO,",obj=",RST_RO["obj"][1])
            if status_RO != "OPTIMAL"
                println("Warning: The RO model did not reach optimality")
                continue
            end
            ratio = abs(RST_RO["obj"][1] - RST_ETO["obj"][1])/abs(RST_ETO["obj"][1])
            # ratio = abs(RST_RO["profit"][1] - RST_ETO["profit"][1])/abs(RST_ETO["profit"][1])
            if ratio > 1e-3
                println("Warning: The RO obj is not equivalent to ETO obj: ETO_Obj=",RST_ETO["obj"][1],",RO_Obj=",RST_RO["obj"][1])
                continue
            end
            RST_RO_this[string("gamma=",gamma)] = RST_RO

            for g_index in 2:length(gamma_list)
                gamma = gamma_list[g_index]
                RST_RO,status_RO = solve_RO_this(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
                # println("gamma = $gamma, RO: status = ",status_RO,",obj=",RST_RO["obj"][1])
                RST_RO_this[string("gamma=",gamma)] = RST_RO
            end
            RST_RO_All["ins=$(ins)"] = RST_RO_this

            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"), Input_Data);
        save(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"), RST_True_All);
        save(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"), RST_ETO_All);
        save(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda)_Mis_coef=$(Mis_coef).jld2"), RST_RO_All);
    end
end

main_process (generic function with 1 method)

### Lambda = 0.001

In [ ]:
lambda = 0.001
Mis_coef = 1.05
main_process(S_train_list,lambda,instances,Mis_coef)         

********** S_train = 100 **********
******* ins = 1*********
******* ins = 2*********


In [16]:
profit_True, profit_ETO, profit_RO = output_results(S_train,lambda,data_dir,instances,true);

S=100,lambda=0.001,obj True:0.5384
S=100,lambda=0.001,obj ETO:0.7036
S=100,lambda=0.001,obj RO:[0.7036 0.6907 0.6768 0.6465 0.6088 0.5678 0.5267]

S=100,lambda=0.001,profit True:0.5384
S=100,lambda=0.001,profit ETO/True:0.6523
S=100,lambda=0.001,profit RO/True:[0.6523 0.6887 0.6887 0.7295 0.7413 0.7946 0.8136]
S=100,lambda=0.001,profit RO/ETO:[1.0 1.0559 1.0559 1.1185 1.1365 1.2182 1.2474]
